In [1]:
# %pip install bm25s
# %pip install spacy
# %pip install -U 'spacy[cuda12x]'
# %pip install rouge_score
# %pip install pysbd

In [2]:
import functions as fct
import bm25s
import numpy as np
import json
import time

c:\Users\Nicolas\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
c:\Users\Nicolas\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


resource module not available on Windows


# BM25

In [5]:
# Définition des chemins vers les fichiers JSON d'entraînement et de développement
train_path = 'SCOTUS/train.json'
dev_path = 'SCOTUS/dev.json'

# Ouverture du fichier d'entraînement
train = fct.open_file(train_path, "json")

# Récupération du document source du premier élément de l'ensemble d'entraînement
# document = train[0]["raw_source"]
document = train

# Récupération des éléments de la cible (faits, question, conclusion)
paragraph_target = (
    train[0]['raw_target']['facts_of_the_case'] +
    train[0]['raw_target']['question'] +
    train[0]['raw_target']['conclusion']
)

train_path = 'data_txt_save/train_0.txt'
train = fct.open_file(train_path, "txt")

document = train


# Segmentation des phrases du document source
sentences = fct.sent_segmentation(document, method='custom_spacy')

# Résumé des phrases en utilisant le modèle BERT
summary = fct.bb25LegalSum(sentences, "bert-base-uncased", 5)

# Évaluation de la qualité du résumé à l'aide de la métrique ROUGE
bb25_rouge = fct.rouge_evaluations(" ".join(summary), paragraph_target)

c:\Users\Nicolas\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\Nicolas\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


In [6]:
bb25_rouge

,Metric,Precision,Recall,F1-Score
0,rouge1,0.333333,0.513966,0.404396
1,rouge2,0.109091,0.168539,0.132450
2,rougeL,0.148551,0.229050,0.180220


In [10]:
# Définition des chemins vers les fichiers JSON d'entraînement et de développement
train_path_json = 'SCOTUS/train.json'
dev_path_json = 'SCOTUS/dev.json'

# Ouverture du fichier d'entraînement JSON
train_json = fct.open_file(train_path_json, "json")

# Récupération du document source et des éléments de la cible (faits, question, conclusion)
document_json = train_json[0]["raw_source"]
paragraph_target_json = (
    train_json[0]['raw_target']['facts_of_the_case'] +
    train_json[0]['raw_target']['question'] +
    train_json[0]['raw_target']['conclusion']
)

# Définir le chemin du fichier texte à traiter
train_path_txt = 'data_txt_save/train_0.txt'
clean_path_txt = 'clean_data_txt_save/train_0.txt'

# Ouverture du fichier d'entraînement TXT
document_txt = fct.open_file(train_path_txt, "txt")
clean_document_txt = fct.open_file(clean_path_txt, "txt")

# Liste des méthodes de segmentation et des modèles à tester
methods = ['nltk', 'spacy', 'pySBD', 'custom_spacy']  # Méthodes de segmentation à tester
model = "bert-base-uncased"  # Modèle de résumé à utiliser
results = {}  # Dictionnaire pour stocker les résultats
execution_times = {}  # Dictionnaire pour stocker les temps d'exécution


# Fonction pour évaluer les modèles
def evaluate_models(document, paragraph_target, file_type):
    for method in methods:
        start_time = time.time()

        # Segmentation des phrases du document source
        sentences = fct.sent_segmentation(document, method=method)
        
        # Résumé des phrases
        summary = fct.bb25LegalSum(sentences, model, 5)
        
        # Évaluation de la qualité du résumé à l'aide de la métrique ROUGE
        bb25_rouge = fct.rouge_evaluations(" ".join(summary), paragraph_target)

        end_time = time.time()
        execution_time = end_time - start_time
        
        # Stockage des résultats dans le dictionnaire
        results[f"{file_type} - {method}"] = bb25_rouge
        execution_times[f"{file_type} - {method}"] = execution_time

# Évaluation des modèles pour le fichier JSON
evaluate_models(document_json, paragraph_target_json, "JSON")

# Évaluation des modèles pour le fichier TXT
evaluate_models(document_txt, paragraph_target_json, "TXT")  # Utilise le même paragraphe cible
evaluate_models(clean_document_txt, paragraph_target_json, "Cleaned TXT")  # Utilise le même paragraphe cible

print("\nTemps d'exécution (en secondes):")
for key, value in execution_times.items():
    print(f"{key}: {value:.4f} secondes")

c:\Users\Nicolas\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\Nicolas\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\Nicolas\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\Nicolas\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when t


Temps d'exécution (en secondes):
JSON - nltk: 31.8896 secondes
JSON - spacy: 160.9200 secondes
JSON - pySBD: 161.8250 secondes
JSON - custom_spacy: 134.4546 secondes
TXT - nltk: 17.3973 secondes
TXT - spacy: 21.1396 secondes
TXT - pySBD: 17.1433 secondes
TXT - custom_spacy: 21.3029 secondes
Cleaned TXT - nltk: 17.5163 secondes
Cleaned TXT - spacy: 19.6493 secondes
Cleaned TXT - pySBD: 22.0497 secondes
Cleaned TXT - custom_spacy: 23.6572 secondes


In [14]:
import pandas as pd
dfs = []

for key, value in results.items():
    df = pd.DataFrame(value)  # Convert the dictionary to a DataFrame
    df['Method'] = key        # Add a 'Method' column
    df['Execution Time'] = execution_times.get(key, None)  # Add the execution time
    dfs.append(df) 

# Combine the DataFrames
final_df = pd.concat(dfs, ignore_index=True)

# Rearrange the columns for better readability
final_df = final_df[['Method', 'Metric', 'Precision', 'Recall', 'F1-Score', 'Execution Time']]

def highlight_min_max_and_lines(df):
    styles = pd.DataFrame('', index=df.index, columns=df.columns)
    
    # Highlight min/max for Precision, Recall, F1-Score, and Execution Time
    for col in ['Precision', 'Recall', 'F1-Score', 'Execution Time']:
        is_max = df[col] == df[col].max()
        is_min = df[col] == df[col].min()
        styles[col] = ['background-color: green' if v else 'background-color: red' if v_min else '' 
                       for v, v_min in zip(is_max, is_min)]
    
    # Add gray borders between methods
    for i in range(1, len(df)):
        if df['Method'].iloc[i] != df['Method'].iloc[i - 1]:
            styles.iloc[i - 1] += 'border-bottom: 2px solid gray'  # Gray line between methods

    return styles

# Apply the combined styling
styled_df = final_df.style.apply(highlight_min_max_and_lines, axis=None)

# Display the styled DataFrame in Jupyter Notebook
styled_df

,Method,Metric,Precision,Recall,F1-Score,Execution Time
0,JSON - nltk,rouge1,0.449275,0.038871,0.071552,31.889611
1,JSON - nltk,rouge2,0.109091,0.009407,0.017321,31.889611
2,JSON - nltk,rougeL,0.184783,0.015987,0.029429,31.889611
3,JSON - spacy,rouge1,0.094203,0.292135,0.142466,160.919978
4,JSON - spacy,rouge2,0.025455,0.079545,0.038567,160.919978
5,JSON - spacy,rougeL,0.076087,0.235955,0.115068,160.919978
6,JSON - pySBD,rouge1,0.086957,0.279070,0.132597,161.825015
7,JSON - pySBD,rouge2,0.010909,0.035294,0.016667,161.825015
8,JSON - pySBD,rougeL,0.050725,0.162791,0.077348,161.825015
9,JSON - custom_spacy,rouge1,0.083333,0.252747,0.125341,134.454624


In [9]:
import pandas as pd

# Création d'une fonction pour formater les données et afficher les résultats
def afficher_resultats_rouge():
    # Données sous forme de dictionnaire
    resultats = {
        "Méthode": ["JSON - nltk", "JSON - spacy", "JSON - pySBD", "TXT - nltk", "TXT - spacy", "TXT - pySBD", "Cleaned TXT - nltk", "Cleaned TXT - spacy", "Cleaned TXT - pySBD"],
        "rouge1_Precision": [0.449275, 0.094203, 0.086957, 0.257246, 0.271739, 0.474638, 0.373188, 0.257246, 0.474638],
        "rouge1_Recall": [0.038871, 0.292135, 0.279070, 0.467105, 0.563910, 0.587444, 0.559783, 0.559055, 0.598174],
        "rouge1_F1-Score": [0.071552, 0.142466, 0.132597, 0.331776, 0.366748, 0.525050, 0.447826, 0.352357, 0.529293],
        "rouge2_Precision": [0.109091, 0.025455, 0.010909, 0.080000, 0.090909, 0.167273, 0.123636, 0.087273, 0.167273],
        "rouge2_Recall": [0.009407, 0.079545, 0.035294, 0.145695, 0.189394, 0.207207, 0.185792, 0.190476, 0.211009],
        "rouge2_F1-Score": [0.017321, 0.038567, 0.016667, 0.103286, 0.122850, 0.185111, 0.148472, 0.119701, 0.186613],
        "rougeL_Precision": [0.184783, 0.076087, 0.050725, 0.126812, 0.126812, 0.202899, 0.181159, 0.126812, 0.192029],
        "rougeL_Recall": [0.015987, 0.235955, 0.162791, 0.230263, 0.263158, 0.251121, 0.271739, 0.275591, 0.242009],
        "rougeL_F1-Score": [0.029429, 0.115068, 0.077348, 0.163551, 0.171149, 0.224449, 0.217391, 0.173697, 0.214141]
    }

    # Création d'un DataFrame pour mettre en forme les résultats
    df = pd.DataFrame(resultats)

    # Affichage des résultats
    print(df.to_string(index=False))

# Appel de la fonction pour afficher les résultats
afficher_resultats_rouge()


            Méthode  rouge1_Precision  rouge1_Recall  rouge1_F1-Score  rouge2_Precision  rouge2_Recall  rouge2_F1-Score  rougeL_Precision  rougeL_Recall  rougeL_F1-Score
        JSON - nltk          0.449275       0.038871         0.071552          0.109091       0.009407         0.017321          0.184783       0.015987         0.029429
       JSON - spacy          0.094203       0.292135         0.142466          0.025455       0.079545         0.038567          0.076087       0.235955         0.115068
       JSON - pySBD          0.086957       0.279070         0.132597          0.010909       0.035294         0.016667          0.050725       0.162791         0.077348
         TXT - nltk          0.257246       0.467105         0.331776          0.080000       0.145695         0.103286          0.126812       0.230263         0.163551
        TXT - spacy          0.271739       0.563910         0.366748          0.090909       0.189394         0.122850          0.126812       0.2631